In [19]:
import pandas as pd

from pyspark.sql import SparkSession

Load combined dataset and removing duplicated rows

In [20]:
df = pd.read_csv('./Twitter_Airline Dataset/combined.csv').drop_duplicates()
df.head()

C:\Users\Jurgen\AppData\Local\Temp\ipykernel_11596\1164425126.py:1: DtypeWarning: Columns (4,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./Twitter_Airline Dataset/combined.csv').drop_duplicates()


,_unit_id,_created_at,_golden,_id,_missed,_started_at,_tainted,_channel,_trust,_worker_id,...,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,681448150,2/25/2015 04:52:40,False,1575073003,NaN,2/25/2015 04:49:12,False,elite,0.8108,31110645,...,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,24/2/2015 11:35,5.700000e+17,NaN,Eastern Time (US & Canada)
1,681448150,2/25/2015 05:22:10,False,1575093916,NaN,2/25/2015 05:19:59,False,prodege,0.8919,1908948,...,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,24/2/2015 11:35,5.700000e+17,NaN,Eastern Time (US & Canada)
2,681448150,2/25/2015 05:24:40,False,1575095606,NaN,2/25/2015 05:20:10,False,clixsense,0.7222,31396033,...,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,24/2/2015 11:35,5.700000e+17,NaN,Eastern Time (US & Canada)
3,681448153,2/25/2015 01:43:24,False,1574913129,NaN,2/25/2015 01:40:59,False,neodev,0.8966,31346116,...,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,24/2/2015 11:15,5.700000e+17,NaN,Pacific Time (US & Canada)
4,681448153,2/25/2015 01:43:34,False,1574913225,NaN,2/25/2015 01:40:56,False,neodev,0.8108,31452413,...,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,24/2/2015 11:15,5.700000e+17,NaN,Pacific Time (US & Canada)


In [21]:
df_deduplicated = df.drop_duplicates(subset=['text', '_id'])
print(df_deduplicated.shape)

(58882, 27)


In [22]:
df_deduplicated.isnull().sum()[df.isnull().sum() != 0]

_missed                   57005
_country                    212
_region                    9680
_city                      9831
negativereason            21286
airline_sentiment_gold    46288
negativereason_gold       48989
tweet_coord               54716
tweet_location            19279
user_timezone             18882
dtype: int64

### Comparing airline_sentiment and airline_sentiment_golden

In [23]:

len(df_deduplicated) == df_deduplicated.airline_sentiment.value_counts().sum()

True

In [24]:
len(df_deduplicated) == df_deduplicated.airline_sentiment_gold.value_counts().sum()

False

In [25]:
# Count the number of matching values between 'airline_sentiment' and 'airline_sentiment_gold'
matching_values_count = df_deduplicated[df_deduplicated['airline_sentiment_gold'].notnull() & (df_deduplicated['airline_sentiment_gold'] == df_deduplicated['airline_sentiment'])].shape[0]

print("Number of matching values:", matching_values_count)
print("Number of rows with non-null 'airline_sentiment_gold':", df_deduplicated['airline_sentiment_gold'].notnull().sum())


Number of matching values: 12107
Number of rows with non-null 'airline_sentiment_gold': 12594


In [26]:
df_deduplicated[(df_deduplicated['airline_sentiment_gold'] != df_deduplicated['airline_sentiment']) & (df_deduplicated['airline_sentiment_gold'].notna())][["airline_sentiment_gold", "airline_sentiment"]]

,airline_sentiment_gold,airline_sentiment
3926,negative,positive
4098,negative,positive
6523,negative,neutral
6921,neutral,positive
6932,neutral,positive
...,...,...
47979,negative,neutral
47983,negative,neutral
47995,negative,neutral
71189,positive,negative


Since there is a discrepency between the airline_sentiment and airline_sentiment_gold, if the airline_sentiment_gold values are present, they will be replaced to fill up the airline_sentiment

### Comparing negativereason and negativereason_gold

In [27]:
df_deduplicated.negativereason.unique()

array([nan, 'badflight', 'CSProblem', 'canttell', 'late', 'airplanestaff',
       'booking', 'lostluggae', 'cancel', 'damagedluggage', 'longlines'],
      dtype=object)

In [28]:
df_deduplicated.negativereason_gold.unique()

array([nan, 'late\r\nairplanestaff', 'late\r\nlostluggae', 'late',
       'CSProblem', 'canttell', 'cancel\r\nCSProblem', 'cancel',
       'badflight', 'lostluggae\r\ndamagedluggage', 'late\r\ncancel',
       'airplanestaff', 'CSProblem\r\nlostluggae',
       'CSProblem\r\ncanttell'], dtype=object)

In [29]:
df_deduplicated.negativereason_gold = df.negativereason_gold.str.strip().str.replace('\r\n', '&')

C:\Users\Jurgen\AppData\Local\Temp\ipykernel_11596\872523559.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduplicated.negativereason_gold = df.negativereason_gold.str.strip().str.replace('\r\n', '&')


In [30]:
df_deduplicated.negativereason_gold.unique()

array([nan, 'late&airplanestaff', 'late&lostluggae', 'late', 'CSProblem',
       'canttell', 'cancel&CSProblem', 'cancel', 'badflight',
       'lostluggae&damagedluggage', 'late&cancel', 'airplanestaff',
       'CSProblem&lostluggae', 'CSProblem&canttell'], dtype=object)

In [41]:
# negativereason_gold split by '&' and use the value in the firsts position as the new value for negativereason1 and the value in the second position as the new value for negativereason2 if it exists
df_deduplicated[['negativereason1', 'negativereason2']] = df_deduplicated.negativereason_gold.str.split('&', expand=True)

# If negativereason1 is null, use the value in negativereason
df_deduplicated.loc[df_deduplicated.negativereason1.isnull(), 'negativereason1'] = df_deduplicated.negativereason

C:\Users\Jurgen\AppData\Local\Temp\ipykernel_11596\1710278856.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduplicated[['negativereason1', 'negativereason2']] = df_deduplicated.negativereason_gold.str.split('&', expand=True)


In [42]:
df_deduplicated.negativereason1.value_counts()

negativereason1
CSProblem         13405
late               6841
canttell           4808
cancel             4014
lostluggae         2371
badflight          2110
airplanestaff      1754
booking            1696
longlines           659
damagedluggage      219
Name: count, dtype: int64

In [44]:
df_deduplicated.negativereason2.value_counts()

negativereason2
CSProblem         604
lostluggae        596
damagedluggage    322
airplanestaff     317
canttell          314
cancel            293
Name: count, dtype: int64

Suggestion: Seperate into 2 columns negativereason1 and negativereason2

In [25]:
df_deduplicated[(df_deduplicated['negativereason_gold'] != df_deduplicated['negativereason']) & (df_deduplicated['negativereason_gold'].notna())][["negativereason_gold", "negativereason"]]

,negativereason_gold,negativereason
3858,late&airplanestaff,late
3859,late&airplanestaff,late
3860,late&airplanestaff,late
3861,late&airplanestaff,late
3862,late&airplanestaff,CSProblem
...,...,...
74269,CSProblem,booking
74300,CSProblem,longlines
74303,CSProblem,booking
101686,CSProblem,late


### Analysing of missing geographical columns - _country, tweet_location and user_timezone

In [26]:
# Replacing the 'airline_sentiment' column with the 'airline_sentiment_gold' column where the latter is not null
df_deduplicated['airline_sentiment'] = df_deduplicated['airline_sentiment_gold'].where(df_deduplicated['airline_sentiment_gold'].notnull(), df_deduplicated['airline_sentiment'])

C:\Users\Jurgen\AppData\Local\Temp\ipykernel_29500\2230009250.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduplicated['airline_sentiment'] = df_deduplicated['airline_sentiment_gold'].where(df_deduplicated['airline_sentiment_gold'].notnull(), df_deduplicated['airline_sentiment'])


In [27]:
df_deduplicated._country.unique()

array(['IND', 'CAN', 'MEX', 'ITA', 'CHL', 'TUR', 'HUN', 'GBR', 'MYS',
       'VEN', 'BRA', 'USA', 'PRT', 'ESP', 'HRV', 'BIH', 'VNM', 'SRB',
       'DEU', 'EGY', 'MKD', 'IDN', 'RUS', 'PHL', 'ARG', 'SGP', 'MAR',
       'ISR', 'BGD', 'ARE', 'PAK', 'ROU', 'URY', 'UGA', 'HKG', 'JPN',
       'FIN', 'NLD', 'CHE', 'GRC', 'PER', 'BEL', 'AUS', 'BGR', 'FRA',
       'EST', 'POL', 'IRL', nan, 'COL', 'BHS', 'SWE', 'AUT', 'ZAF', 'LVA',
       'DOM', 'DZA', 'TZA', 'NPL', 'JOR', 'SLV', 'CYP', 'PRI', 'UKR',
       'KEN', 'SVK', 'KWT'], dtype=object)

In [28]:
df_deduplicated._region.unique()

array(['2', 'ON', '19', '18', '12', '34', '5', nan, '14', '27', 'OH',
       '17', '56', '54', '21', 'NV', '51', '44', '13', '1', '52', '25',
       '39', 'H9', '41', '67', '22', '16', '66', 'B7', '0', 'IL', '10',
       'D9', '4', '45', 'M3', '81', '8', 'BC', '38', '37', '7', '30',
       '47', 'L9', '15', '48', 'TX', 'MA', 'K7', '36', 'C9', '49', '6',
       'G6', 'NS', '28', '35', '11', '31', '40', 'B4', '78', '3', 'MD',
       'K1', 'E3', 'M9', '58', '23', 'CA', '20', 'F2', 'V1', '55', '29',
       '72', 'AB', '98', 'B3', 'AK', '77', '24', 'K2', '9', '68', '26',
       'B8', '42', 'L6', 'FL', 'C3', 'KS', 'AL', '71', 'A3', 'E4', '74',
       'MI', '43'], dtype=object)

In [29]:
df_deduplicated.tweet_location.unique()

array([nan, 'Lets Play', 'San Francisco CA', ...,
       '??T: 41.498967,2.186957', 'Ciudad Ju??rez MX. [CJS/MMCS]',
       "??? Chicago'ish ???"], dtype=object)

In [30]:
df_deduplicated.user_timezone.unique()

array(['Eastern Time (US & Canada)', 'Pacific Time (US & Canada)',
       'Central Time (US & Canada)', 'America/New_York',
       'Atlantic Time (Canada)', 'Quito', nan,
       'Mountain Time (US & Canada)', 'Vienna', 'Caracas', 'Kuala Lumpur',
       'Brisbane', 'Arizona', 'London', 'Tehran', 'Alaska', 'Sydney',
       'Irkutsk', 'Santiago', 'Amsterdam', 'Tijuana', 'Abu Dhabi',
       'Central America', 'Edinburgh', 'Jerusalem', 'Hawaii', 'Paris',
       'Guam', 'New Delhi', 'Stockholm', 'America/Chicago', 'Berlin',
       'Madrid', 'Athens', 'Brussels', 'Taipei', 'Rome', 'Beijing',
       'Mexico City', 'Bern', 'Singapore', 'Indiana (East)', 'Melbourne',
       'Saskatchewan', 'Casablanca', 'Brasilia', 'Kyiv', 'Bucharest',
       'Greenland', 'Prague', 'New Caledonia', 'Bogota', 'Seoul',
       'Sarajevo', 'Wellington', 'Bangkok', 'Warsaw', 'Copenhagen',
       'Hong Kong', 'Guadalajara', 'Mid-Atlantic', 'Mazatlan',
       'Buenos Aires', 'America/Los_Angeles', 'Dublin', 'Lisbon',
 

In [31]:
df_deduplicated._city.unique()

array(['Hyderabad', 'Toronto', 'Monterrey', 'Bastia', 'Santiago',
       'Istanbul', 'Budapest', nan, 'Kuala Lumpur', 'São Paulo',
       'Fairfield', 'San Nicolás De Los Garza', 'Vila Nova De Gaia',
       'Barcelona', 'Mora', 'Zagreb', 'Doboj', 'Reno',
       'Las Torres De Cotillas', 'Hanoi', 'Modugno', 'Zenica', 'Zaragoza',
       'Caracas', 'Torrelavega', 'London', 'Skopje', 'Nguyên',
       'Querétaro', 'Mumbai', 'Saint Petersburg', 'Mollet Del Vallès',
       'Cebu', 'Punta Alta', 'Singapore', 'Sarajevo', 'Mount Vernon',
       'Turin', 'Macerata', 'Pune', 'Mandaluyong', 'Jakarta',
       'Casablanca', 'Bridgwater', 'Tel Aviv', 'Salvador', 'Udupi', 'Hue',
       'Dhaka', 'Abu Dhabi', 'Karachi', 'Malang', 'Cluj-napoca',
       'Burnaby', 'Vancouver', 'Montevideo', 'Makassar', 'Bokaro',
       'Loures', 'Kampala', 'Champa', 'Kottayam', 'Chennai', 'Semarang',
       'Speyer', 'Ajax', 'Tangerang', 'San Antonio', 'Central District',
       'Sheffield', 'Lahore', 'Petaling Jaya', 'Spl

In [32]:
df_deduplicated.tweet_coord.nunique()
print("Perccentage of 'tweet_coord' in the df:", df.tweet_coord.nunique()/len(df))

Perccentage of 'tweet_coord' in the df: 0.0014925319584817793


As there are too many missing values in the tweet_coord column, it will be best to omit the column

In [33]:
df_deduplicated[df_deduplicated.tweet_location == "New York City"][["_country", "_region", "_city", "tweet_location", "user_timezone", "tweet_coord"]]

,_country,_region,_city,tweet_location,user_timezone,tweet_coord
594,PAK,4,Lahore,New York City,Eastern Time (US & Canada),NaN
595,IND,2,Hyderabad,New York City,Eastern Time (US & Canada),NaN
596,PHL,47,San Antonio,New York City,Eastern Time (US & Canada),NaN
921,TUR,21,Diyarbakir,New York City,Eastern Time (US & Canada),NaN
922,CAN,ON,Ajax,New York City,Eastern Time (US & Canada),NaN
...,...,...,...,...,...,...
82835,VNM,44,Hanoi,New York City,NaN,NaN
82836,IND,2,Hyderabad,New York City,NaN,NaN
83605,MEX,NaN,NaN,New York City,Eastern Time (US & Canada),NaN
83606,IND,16,Mumbai,New York City,Eastern Time (US & Canada),NaN


In [34]:
df_deduplicated[df_deduplicated._country == "IND"][["_country", "_region", "_city", "tweet_location", "user_timezone", "tweet_coord"]]

,_country,_region,_city,tweet_location,user_timezone,tweet_coord
0,IND,2,Hyderabad,NaN,Eastern Time (US & Canada),NaN
13,IND,2,Hyderabad,NaN,Pacific Time (US & Canada),NaN
14,IND,NaN,NaN,NaN,Pacific Time (US & Canada),NaN
15,IND,NaN,NaN,NaN,Pacific Time (US & Canada),NaN
31,IND,NaN,NaN,1/1 loner squad,Eastern Time (US & Canada),NaN
...,...,...,...,...,...,...
110060,IND,2,Hyderabad,NYC,Santiago,NaN
110234,IND,16,Mumbai,Atlanta Ga && Traveling,NaN,NaN
110510,IND,2,Hyderabad,?????SoFla?????,Atlantic Time (Canada),NaN
110740,IND,NaN,NaN,NaN,NaN,NaN


In [35]:
df_deduplicated[df_deduplicated._country.isnull() == True][["tweet_location", "user_timezone"]]

,tweet_location,user_timezone
585,New York City,Eastern Time (US & Canada)
1915,Seattle,Pacific Time (US & Canada)
2054,Seattle,Pacific Time (US & Canada)
2074,London baby,NaN
2324,NYC,Central Time (US & Canada)
...,...,...
104559,"Washington, DC",Eastern Time (US & Canada)
108196,"Wichita Falls, Texas",NaN
109343,SNY in NYC.,Eastern Time (US & Canada)
109686,ATX,Central Time (US & Canada)


Due to a heavy inconsistency and inaccuracy of data in the geographical data column, it is not possible to match the missing values